In [5]:
import matplotlib
%matplotlib qt5
from matplotlib.patches import Circle
import scipy.sparse as sp
import scipy.sparse.linalg as spl
from scipy.integrate import simps
import numpy as np
import matplotlib.pylab as plt
import matplotlib.colors as colors
import time
import sys
from astropy import units as u
from astropy.constants import R # Ideal gas constant                                                                                                                                                               
import astropy

5.2.1


# Parameters

## Model parameters

In [6]:
unitsL = u.Unit('cm')                                                                                                                                                                                              
unitsT = u.Unit('ms')                                                                                                                                                                                              
                                                                                                                                                                                                                   
#R = R.to((unitsL**3) * u.torr / u.K / u.mol) # Gas constant                                                                                                                                                       
f_CRA = 49.34 * u.uL/u.min                                                                                                                                                                                         
w = 1*u.um # 1 micron                                                                                                                                                                                              
U = 2400 * u.um*u.Unit('micron')/u.s # Permeability in um^2/s                                                                                                                                                      
D = 1800 * u.um*u.Unit('micron')/u.s # Diffusion in um^2/s                                                                                                                                                         
c0 = 50 * u.torr                                                                                                                                                                                                   
kt = 4.5 / u.min *0                                                                                                                                                                                                
Tb = 309.25 * u.K    # Blood temperature, 36.1*C in Kelvin                                                                                                                                                         
spacing = u.Quantity([5, 5, 10], 'micron')                                                                                                                                                                         
                                                                                                                                                                                                                   
print(f"""Simulation coefficients:                                                                                                                                                                                 
\tInlet concentration: {c0=}                                                                                                                                                                                       
\tWall thickness: {w=}                                                                                                                                                                                             
\tDiffusion: {D=}                                                                                                                                                                                                  
\tWall permeability: {U=}                                                                                                                                                                                          
\tTissue consumption: {kt=}                                                                                                                                                                                        
\tInlet blood flow (may not be used): {f_CRA=}                                                                                                                                                                     
""")       

c0 = c0/R/Tb                                                                                                                                                                                                       
w = w.to_value(unitsL)                                                                                                                                                                                       
f_CRA = f_CRA.to_value(unitsL**3 / unitsT)                                                                                                                                                                   
U = U.to_value(unitsL**2 / unitsT)                                                                                                                                                                           
D = D.to_value(unitsL**2 / unitsT)                                                                                                                                                                           
kt = kt.to_value(unitsT**-1)                                                                                                                                                                                 
c0 = c0.to_value(u.mol / (unitsL**3))

Simulation coefficients:                                                                                                                                                                                 
	Inlet concentration: c0=<Quantity 50. Torr>                                                                                                                                                                                       
	Wall thickness: w=<Quantity 1. um>                                                                                                                                                                                             
	Diffusion: D=<Quantity 1800. micron um / s>                                                                                                                                                                                                  
	Wall permeability: U=<Quantity 2400. micron um / s>                                                      

## Geometric parameters

In [3]:
r = (10*u.um).to_value(unitsL) # Radius
l = (100*u.um).to_value(unitsL)

x = np.arange(0,l+dx,dx)
y = np.arange(0,50*r+dy,dy) 
L,H = x.max(), y.max()
X,Y = np.meshgrid(x,y)
nx = x.size
ny = y.size
# Corners of the vessel

i0, i1 = int((y.max()/2-r)/dx), int((y.max()/2+r)/dx) # int(7*ny/16), int(9*ny/16)
j0, j1 = 0*int(4*nx/16), nx #int(12*nx/16)

# i0, i1 = int(7*ny/16), int(9*ny/16)
# j0, j1 = int(4*nx/16), int(12*nx/16)
print(f"{i0*dy=} {i1*dy=} {j0*dx=} {j1*dx=}")

i0*dy=0.12000000000000001 i1*dy=0.13 j0*dx=0.0 j1*dx=0.1004


In [33]:
coeffs = [dx/dy*D, dy/dx*D, 0, dy/dx*D, dx/dy*D]
coeffs[2] = -sum(coeffs)# - k*dx*dy
offsets = [-nx, -1, 0, 1, nx]
print(f"{nx=} {ny=} {nx*ny=}")

nx=251 ny=626 nx*ny=157126


# Testing reaction diffusion sub-system

In [5]:
# M = sp.diags(coeffs, [-nx, -1, 0, 1, nx], shape=(nx*ny, nx*ny), format='lil')
# for j in range(y.size):
#     for i in range(x.size):
#         cell = j*nx + i
#         if i==0:
#             M[cell,cell] += coeffs[1]
#             if cell-1 >= 0:
#                 M[cell, cell-1] = 0.0
#         if i==nx-1:
#             M[cell, cell] += coeffs[3]
#             if cell+1 < M.shape[1]:
#                 M[cell, cell+1] = 0.0
#         if j==0:
#             M[cell, cell] += coeffs[0]
#         if j==ny-1:
#             M[cell, cell] += coeffs[4]


In [6]:
# u = np.zeros(X.size)
# deletecells = sp.eye(M.shape[0]).tolil()
# for j, yj in enumerate(y):
#     for i, xi in enumerate(x):
#         if (xi-0.5)**2 + (yj-0.5)**2 < 0.2**2:
#             cell = i + j*nx
#             deletecells[cell, cell] = 0.0
#             u[cell]= 1.0
u = np.zeros_like(X)
u[i0:i1, j0:j1] = 1.0
u = u.reshape((-1,))
# for i in np.nonzero(u):
#     deletecells[i,i] = 0

# print(f"{u.mean()=}, {u.min()=}, {u.max()=} {u.sum()=}")
# fig, ax = plt.subplots()
# c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
# fig.colorbar(c, ax=ax)
# plt.show()

In [7]:
# M = deletecells.dot(M)
# A = -dt*M + sp.eye(M.shape[0])
# A = A.tocsr()
# for k in range(10):
#     u = spl.spsolve(A, u)
#     print(f"{u.mean()=}, {u.min()=}, {u.max()=}")

In [8]:
# fig, ax = plt.subplots()
# c = ax.pcolormesh(X,Y, u.reshape(X.shape), cmap='RdBu', vmin=u.min(), vmax=u.max())
# fig.colorbar(c, ax=ax)
# circ = Circle((0.5,0.5), 0.2, fill=False, linestyle='--', linewidth=3)
# ax.add_patch(circ)
# plt.show()

# Build convection matrix

In [9]:
nPts = j1-j0
nVol = nx*ny
C = sp.diags([-f/dx,f/dx], [-1,0], shape=(nPts, nPts+nVol), format='lil')
#C[nPts-1,nPts] = 0
#C[0,1] = 0 # Will be a fixed inlet concentration
C = C.tocsr() 
# Add empty rows at the bottom of C to make it the right size
indptr = C.indptr.tolist()
for i in range(nVol):
    indptr.append(indptr[-1])
C = sp.csr_matrix((C.data, C.indices, indptr), shape=(nPts+nVol, nPts+nVol))

# Connectivity matrices

## Build matrix equating vascular voxel concentrations to nodal concentration

In [10]:
NodesToEndothelialCells = sp.lil_matrix((nPts, nPts+nVol))
for node, j in enumerate(range(j0, j1)):
    for i in [i0-1,i1]:
        cell = i*nx + j
        # NodesToEndothelialCells[node, node] += -1
        NodesToEndothelialCells[node, node] = -1
        NodesToEndothelialCells[node, nPts+cell] = 1
for i in range(i0, i1):
    for node, j in zip([0, nPts-1],[j0-1,j1]):
        if j<0 or j>nx-1:
            continue
        cell = i*nx + j
        NodesToEndothelialCells[node, nPts+cell] = 1
        NodesToEndothelialCells[node, node] = -1        
        # NodesToEndothelialCells[node, node] += -1        

# for node in range(nPts):
#     nAdjacentEndothelialCells = NodesToEndothelialCells[node, nPts:].nonzero()[0].size
#     NodesToEndothelialCells[node, nPts:] /= nAdjacentEndothelialCells   

## Labels

In [11]:
Arr = NodesToEndothelialCells.toarray()
labels = np.zeros(nVol)
for j in range(j0, j1):
    for i in range(i0, i1):
        labels[i*nx+j] = 1

for node in range(nPts):
    labels[np.nonzero(Arr[node, nPts:])] = 2 

# fig, ax = plt.subplots()
# #c = ax.pcolormesh(X,Y,labels.reshape(X.shape), cmap='RdBu')#, edgecolor='k')
# c = ax.pcolormesh(labels.reshape(X.shape)[i1:,:], cmap='RdBu')#, edgecolor='k')
# plt.colorbar(c, ax=ax, label='0 for tissue, 1 for vessel, 2 for endothelial cells')

## Matrix linking vascular voxels with vascular node

In [12]:
C4 = sp.lil_matrix((nVol, nPts))
I4 = np.zeros(nVol)
uVascularNodeToVoxel = np.zeros(nVol)
for node,j in enumerate(range(j0, j1)):
    for i in range(i0, i1):
        cell = i*nx+j
        C4[cell,node] = 1
        I4[cell] = 1
        uVascularNodeToVoxel[cell] = node+1
I4 = sp.diags([I4], [0]).tocsr()

# Check the loss
cv = np.ones(nPts) # Vascular nodal oxygen
print(np.any(I4.dot(u)-C4.dot(cv)!=0.0)) # Return False if each voxel's oxygen is equal to its corresponding node's oxygen 

# fig, ax = plt.subplots()
# c = ax.pcolormesh(uVascularNodeToVoxel.reshape(X.shape), cmap='RdBu', edgecolor='k')
# plt.colorbar(c, ax=ax, label='Linked to node')

False


# Build mass exchange between vessels and tissue

In [13]:
# V = dx#x.max()*y.max() # *dx*dy
A = 2*np.pi*r*L
Gamma = sp.diags([np.ones(nPts)*U*A/w], [0], shape=(nPts, nPts))
MassExchange = -NodesToEndothelialCells.T.dot(Gamma).dot(NodesToEndothelialCells)

# fig, ax = plt.subplots()
# c = ax.pcolormesh(abs(MassExchange.toarray()), cmap='RdBu')
# fig.colorbar(c, ax=ax)
# plt.show()

### Check if endothelial cells are connected to the right vascular nodes

In [14]:
Arr = NodesToEndothelialCells.toarray()
uCheckConnectionEndoToNode = uVascularNodeToVoxel
for node in range(nPts):
    uCheckConnectionEndoToNode[np.nonzero(Arr[node, nPts:])] = node+1

# fig, ax = plt.subplots()
# c = ax.pcolormesh(uCheckConnectionEndoToNode.reshape(X.shape), cmap='RdBu', edgecolor='k')
# plt.colorbar(c, ax=ax)
# plt.show()

# Reaction-Diffusion matrices

## Diffusion part (including BC)

In [15]:
coeffs = [dx/dy*D, dy/dx*D, 0, dy/dx*D, dx/dy*D]
coeffs[2] = -sum(coeffs)
offsets = [-nx, -1, 0, 1, nx]
print(f"{nx=} {ny=}")
M = sp.diags(coeffs, [-nx, -1, 0, 1, nx], shape=(nx*ny, nx*ny), format='lil')

# Tissue external boundary conditions
for j in range(y.size):
    for i in range(x.size):
        cell = j*nx + i
        if i==0:
            M[cell,cell] += coeffs[1]
            if cell-1 >= 0:
                M[cell, cell-1] = 0.0
        if i==nx-1:
            M[cell, cell] += coeffs[3]
            if cell+1 < M.shape[1]:
                M[cell, cell+1] = 0.0
        if j==0:
            M[cell, cell] += coeffs[0]
        if j==ny-1:
            M[cell, cell] += coeffs[4]

nx=251 ny=626


## Make the consumption rate matrix and diffusion matrix (cells with no consumption/no diffusion need to be excluded)

In [16]:
deleteVascularCells = np.ones(nVol)
offsets.pop(2)
coeffs.pop(2)
V = x.max()*y.max()*r 
R = np.ones(nVol)*k*V 
for i,label in enumerate(labels):
    if label==1:
        deleteVascularCells[i] = 0
    if label==2:
        #R[i] /= k # Je me souviens pas pourquoi j'ai mis ca 
        # R[i] = 0
        # Tissue internal (endothelial to vascular voxels) boundary conditions.
        # Get rid of diffusion from vascular voxels to endothelial voxels
        for offset, coeff in zip(offsets, coeffs):
            if i+offset>=0:
                try:
                    if label[i+offset] == 1:
                        M[i, i+offset] = 0.0
                        M[i, i] -= coeff
                except:
                    pass

deleteVascularCells = sp.diags([deleteVascularCells], [0])

R = sp.diags([R], [0])
R = deleteVascularCells.dot(R)
M = deleteVascularCells.dot(M)

In [17]:
print(nPts, nVol)
print(f"{MassExchange.shape=} {C.shape=} {M.shape=} {R.shape=} {C4.shape=} {I4.shape=}")
print(MassExchange.nnz, labels[labels>0].size)

251 157126
MassExchange.shape=(157377, 157377) C.shape=(157377, 157377) M.shape=(157126, 157126) R.shape=(157126, 157126) C4.shape=(157126, 251) I4.shape=(157126, 157126)
2259 6777


# Combine and reshape sub-matrices before combining them

In [18]:
# Equations associating vascular cells and nodes
M4 = sp.vstack([sp.csr_matrix((nPts, nPts+nVol)), sp.hstack([-C4, I4])])
# Reaction diffusion equations
Md = sp.vstack([sp.csr_matrix((nPts, nPts+nVol)), sp.hstack([sp.csr_matrix((nVol, nPts)), M-R])])
print(f"{MassExchange.shape=} {C.shape=} {Md.shape=} {M4.shape=}")

start_time = time.time()
K = C.tocsr() - MassExchange.tocsr() + Md.tocsr() + M4.tocsr()
K = sp.hstack([sp.csr_matrix((nPts+nVol,1)), K])
K = sp.vstack([sp.csr_matrix((1,nPts+nVol+1)), K])
print("--- %s seconds for matrix addition ---" % (time.time() - start_time))
# fig = plt.figure()
# plt.spy(K)

MassExchange.shape=(157377, 157377) C.shape=(157377, 157377) Md.shape=(157377, 157377) M4.shape=(157377, 157377)
--- 0.013117074966430664 seconds for matrix addition ---


# Build the RHS vector

In [19]:
b = np.zeros(K.shape[0])
# I = np.ones(nPts+nVol)
# I[:nPts] = 0
# K = sp.diags([I],[0], shape=K.shape).dot(K) 
# for i in range(nPts):
#     K[i,i] = 1
#     b[i] = c0

# 
b[0] = c0 # Inlet blood O2 concentration
# 
I = np.ones(K.shape[0])
I[0] = 0
K = sp.diags([I],[0]).dot(K)
K[0,0] = 1.0
K[1,0] = -f/dx
sp.save_npz('K.npz', K)
np.savetxt('b.npz', b)

/usr/lib/python3/dist-packages/scipy/sparse/_index.py:103: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


# Solve the system

### With scipy sparse

In [20]:
u = spl.spsolve(K, b)

In [21]:
#u *= Rg*Tb
ub, ut = u[:nPts+1], u[nPts+1:]

### Save the results

In [22]:
cellStart = i0*nx+j0
endDistance = 3*r
cellEnd   = cellStart + int(endDistance/dy)*nx + nx-1 # +nx-1 to end on the right side of the domain
np.savetxt("ct_voxel.txt", ut[cellStart:cellEnd+1])
np.savetxt("X_voxel.txt", X[i0:i0+int(endDistance/dy)+1:])
np.savetxt("Y_voxel.txt", Y[i0:i0+int(endDistance/dy)+1, :])
np.savetxt("cv_voxel.txt", ub)

## Plot

In [23]:
fig, ax = plt.subplots(1,2, figsize=(12,6))
ax = ax.ravel()
## Linear plot
#c = ax.pcolormesh(ut.reshape(X.shape), cmap='RdBu')#, vmin=u.min(), vmax=u.max())
c = ax[0].contourf(X,Y, Rg*Tb*ut.reshape(X.shape), cmap='RdBu', levels=50)
#c = ax[0].contourf(X[i1:,:], Y[i1:,:], ut.reshape(X.shape)[i1:,:], cmap='RdBu', levels=50)#, vmin=0, vmax=2)
## Log plot
# c = ax.pcolormesh(ut.reshape(X.shape), cmap='RdBu', 
#                   norm=colors.LogNorm(vmin=u.min(), vmax=u.max()))
fig.colorbar(c, ax=ax[0], extend='max')

ax[1].plot(np.arange(-1,j1-j0)*dx, ub, label='Vascular nodes O2 concentration')

ct = ut[labels!=1].reshape(-1, nx)
cv = ub[1:]
oxygenConsummed = D*(ut[i1*nx+1:i1*nx+1+nx]-ut[-nx:]).sum()*dx/H #k*simps([simps(ct_x, dx=dx) for ct_x in ct], dx=dy)/(Rg*Tb)
oxygenExtracted = (c0-cv[-1])/c0*100 # U*A/w*simps(cv-ct[0,:], dx=dx*2*np.pi*r)/f /(Rg*Tb)
totalTissueOxygen = ct.sum()*x.max()*y.max() /(Rg*Tb)

plt.suptitle(f"""Oxygen consummed [mol/mm^3/ms]: {oxygenConsummed}\
    Oxygen extraction fraction [%]: {oxygenExtracted}\n
    Oxygen inflow [mol/ms]: {c0*f}\
    Oxygen extracted [mol/ms]: {oxygenExtracted*c0*f}""")

plt.legend()
plt.show()

In [24]:
print(ut.mean()*Rg*Tb, ut.max()*Rg*Tb, ut.min()*Rg*Tb)

1.7420694748391774 39.99999347747904 3.6646379171758505e-71


In [25]:
# fig, ax = plt.subplots()
# c = ax.pcolormesh(K[:nPts, :].toarray(), cmap='RdBu')#, edgecolor='k')
# plt.colorbar(c, ax=ax)
# ax.set_aspect(nVol/nPts)
print('Tissue:', ut[labels!=1].mean(), ut[labels!=1].min(), ut[labels!=1].max())
print('Blood: ', ub.mean(), ub.min(), ub.max())

Tissue: 7.81316725191354e-12 1.9002697546080784e-81 9.959647149177153e-10
Blood:  2.0741267324602013e-09 2.074084286924618e-09 2.0741691785719655e-09


In [26]:
# fig = plt.figure()
# plt.plot(np.arange(j1-j0)*dx, ub, label='Vascular nodes O2 concentration')
# plt.hlines(c0*Rg*Tb, 0, (j1-j0)*dx, colors='k', linestyles='--', label='Inlet concentration')
# plt.legend()

# Verify code using forged analytical solutions

In [27]:
# def uv_exact(x):
#     return c0*np.exp(-x*U*A/(w*f))

# def ut_exact(x,y):
#     return np.sin(np.pi*x/2/L) + np.sin(np.pi*y/2/H)

# def gt(x,y):
#     return ( -D*( (np.pi/2/L)**2 ) * np.sin(np.pi*x/2/L) - D*( (np.pi/2/H)**2 )*np.sin(np.pi*y/2/H) 
#             + (U*A/w)*( uv_exact(x) - ut_exact(x,y) ) - k*V* ut_exact(x,y) )  

# def gv(x):
#     return U*A*ut_exact(x,0)/w

## Build the RHS vector

In [28]:
# b = np.zeros(nPts+nVol)
# b[0] = c0 # Inlet blood O2 concentration

# xv = np.arange(j0+1,j1)*dx
# b[1:nPts] = gv(xv)

# for j in range(y.size):
#     yj = j*dy
#     for i in range(x.size):
#         xi = i*dx
#         if labels[j*nx+i] != 1:
#             b[nPts+j*nx+i] = -gt(xi, yj)

## Solve the system

In [29]:
# u = spl.spsolve(K, b)*Rg*Tb
# ub, ut = u[:nPts], u[nPts:]

## Plot

In [30]:
# fig, ax = plt.subplots(1,2, figsize=(12,4))
# ax = ax.ravel()
# ## Linear plot
# #c = ax.pcolormesh(ut.reshape(X.shape), cmap='RdBu')#, vmin=u.min(), vmax=u.max())
# c = ax[0].contourf(X,Y, ut.reshape(X.shape)-ut_exact(X,Y), cmap='RdBu', levels=50)
# ## Log plot
# # c = ax.pcolormesh(ut.reshape(X.shape), cmap='RdBu', 
# #                   norm=colors.LogNorm(vmin=u.min(), vmax=u.max()))
# fig.colorbar(c, ax=ax[0], extend='max')

# ax[1].plot(np.arange(j1-j0)*dx, ub, label='Vascular nodes O2 concentration')
# plt.legend()
# plt.show()

In [31]:
# errt = (ut-ut_exact(X,Y).ravel())/c0
# errt[labels==1] = 0
# plt.contourf(X,Y, errt.reshape(X.shape), cmap='RdBu')
# plt.colorbar()

In [32]:
# fig, ax = plt.subplots(1,2, figsize=(12,4))
# ax = ax.ravel()
# ## Linear plot
# #c = ax.pcolormesh(ut.reshape(X.shape), cmap='RdBu')#, vmin=u.min(), vmax=u.max())
# c = ax[0].contourf(X,Y, ut_exact(X,Y), cmap='RdBu', levels=50)
# ## Log plot
# # c = ax.pcolormesh(ut.reshape(X.shape), cmap='RdBu', 
# #                   norm=colors.LogNorm(vmin=u.min(), vmax=u.max()))
# fig.colorbar(c, ax=ax[0], extend='max')

# ax[1].plot(np.arange(j1-j0)*dx, uv_exact(np.arange(j1-j0)*dx), label='Vascular nodes O2 concentration')
# plt.legend()
# plt.show()